# Visualização de camadas em tempo real

In [1]:
import math

import cv2
import numpy as np
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from labels import labels

### Carregar a rede
Como exemplo iremos utilizar a Resnet50

In [2]:
model = load_model("./data/resnet50.h5")
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
___________________________________________________________________________________________

### Escolha da camada para visualizar
Através do summary podemos escolher alguma camada da rede para nós visualizarmos, camadas mais rasas nos mostraram caracteristicas mais simples e camadas mais densas, caracteristicas mais abstratas.

In [3]:
layer_level = "conv1_relu"
output_layers = [model.get_layer(layer_level).output, model.layers[-1].output]
model_output_layers = Model(model.input, output_layers)

### Executar video

In [4]:
cap = cv2.VideoCapture(0)

Função responsável por gerar a concatenaças dos mapas de caracteristicas da rede

In [5]:
def create_layer_image_sprite(layer_output):
    normalized = layer_output - layer_output.min()
    normalized /= normalized.max()
    normalized *= 255
    normalized = np.uint8(normalized)

    sprite_shape = math.ceil(normalized.shape[2] ** .5)

    sprite = None
    for i in range(sprite_shape):
        row = None
        for j in range(sprite_shape):
            neuron_idx = i * sprite_shape + j
            if neuron_idx >= normalized.shape[2]:  # Caso tenha acabado os filtros
                neuron_output = np.zeros((normalized.shape[0], normalized.shape[1]), dtype=np.uint8)
            else:
                neuron_output = normalized[..., neuron_idx]

            # concatena a linha com as imagens de cara neurônio
            if row is None:
                row = neuron_output
            else:
                row = cv2.hconcat([row, neuron_output])

        # concatena as linhas de imagens
        if sprite is None:
            sprite = row
        else:
            sprite = cv2.vconcat([sprite, row])

    return sprite

### Predição
Predição em tempo real

In [6]:
while True:
    ret, image = cap.read()
    if not ret:
        break
    image = cv2.resize(image, (224, 224))
    blob = preprocess_input(image)
    blob = blob[np.newaxis]
    pred = model_output_layers.predict(blob)

    layer = pred[0][0]
    pred = pred[1][0]

    sprite_output = create_layer_image_sprite(layer)
    sprite_output = cv2.resize(sprite_output, (600, 600))
    sprite_output = cv2.cvtColor(sprite_output, cv2.COLOR_GRAY2BGR)

    input_image_col = np.ones((sprite_output.shape[0], image.shape[1], 3), dtype=np.uint8) * 128
    input_image_col[:image.shape[0], :image.shape[1]] = image
    sprite_output = cv2.hconcat([input_image_col, sprite_output])

    font = cv2.FONT_HERSHEY_PLAIN
    cv2.putText(sprite_output, f"Label: {labels[pred.argmax()].split(',')[0]}", (5, image.shape[0] + 30), font, 1.5, (70, 70, 70), 2)
    cv2.putText(sprite_output, f"Conf.: {pred.max():.2%}", (5, image.shape[0] + 60), font, 1.5, (70, 70, 70), 2)

    cv2.imshow("sprite", sprite_output)
    key = cv2.waitKey(1)
    if key == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()